## Data acquisition

### Public data

We obtained from the Lausanne office of cadastre ftp access to cadastral data. It is a collection of ESRI shapefiles, describing roads, buildings, parcels, trees, waterbodies, and others.
Each shapefile is a collection of features, and each feature has an associated geometry (e.g. the shape of a land parcel) and associated attributes ( e.g. the commune responsible for the parcel, the parcel number).

We can explore this dataset by using GIS software that supports shapefiles. We used QGIS to 
We hoped to find an attribute describing the parcel owner in the parcel shapefile layer, but it wasn't there.
We had to resort to web scraping to recover this attribute.

### Web scraping

We wanted to associate each parcel in Lausanne to an owner. To do this, we divided Lausanne's surface in rectangles, and requested parcel informations for these rectangles to a service exposing the owners name.
The code for the scraping is in `scrape.py`.

## Data munging

The result of this scraping is a set of 400 xml files, each containing parcel information for a geographical rectangle.

In [8]:
!head -n 20 scraped/534810.4210526316_155847.0_535161.3710526315_155589.0.xml

<?xml version='1.0' encoding="UTF-8" ?>
<wfs:FeatureCollection
   xmlns:ms="http://mapserver.gis.umn.edu/mapserver"
   xmlns:gml="http://www.opengis.net/gml"
   xmlns:wfs="http://www.opengis.net/wfs"
   xmlns:ogc="http://www.opengis.net/ogc"
   xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
   xsi:schemaLocation="http://mapserver.gis.umn.edu/mapserver http://map.lausanne.ch/prod/mapserv?SERVICE=WFS&amp;VERSION=1.1.0&amp;REQUEST=DescribeFeatureType&amp;TYPENAME=feature:bdcad_bf_parc_pol&amp;OUTPUTFORMAT=text/xml;%20subtype=gml/3.1.1  http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd">
      <gml:boundedBy>
      	<gml:Envelope srsName="EPSG:21781">
      		<gml:lowerCorner>534766.520000 155512.530000</gml:lowerCorner>
      		<gml:upperCorner>535272.590000 155972.950000</gml:upperCorner>
      	</gml:Envelope>
      </gml:boundedBy>
    <gml:featureMember>
      <ms:bdcad_bf_parc_pol gml:id="bdcad_bf_parc_pol.6341399">
        <gml:boundedBy>
        	<gml:E

### From XML to a single json

We use the `xq` and `jq` programs to extract only the features we care about from the XML and to save them as a list of objects in a json file.
[`multiple_xml_to_single_json.sh`](/edit/multiple_xml_to_single_json.sh) is a small script leveraging the expressiveness of `jq` to efficiently concatenate the XML files into a single json, while also discarding all the attributes we have no interest in.

### cleaning the owners json
This json has duplicate entries, entries concerning other communes than lausanne, entries with missing owners.
Also json is not the best format for tabular data. This cleaning is done in the script [`owner_json_to_clean_csv.py`](/edit/owner_json_to_clean_csv.py).

### joining the owners data with the cadastre shapefiles

The result of the previous preprocessing steps is a csv file with three columns: commune number, parcel number, and owner name. We would like to add the owner name to the attributes of the parcels shapefile that we obtained from "Office du Cadastre". To do so, we import the csv and the shapefile in QGIS, and we join this two "tables" by parcel number. The resulting geographical layer contains all the geographical features representing the parcels, and additionally the owner name for each parcel. We can now export this layer as a GeoJSON, and continue our exploration in a jupyter notebook.